In [5]:
import ujson
import gzip
import numpy as np
import math
from os.path import isfile, isdir
from struct import pack, unpack
from os import makedirs
import os
from struct import pack
import itertools
import scipy.sparse, scipy.io
import intbitset
import pickle as pickle

In [8]:
fname = 'C:\\vk\\group_data.bin'
invfname = 'C:\\vk\\group_data_inv.bin'

In [9]:
n_nodes = 350000000

In [10]:
memcount = np.zeros(n_nodes, dtype=np.int)
buffer_size = 1024
last_node = 1
cnt = 0
with open(fname, 'rb') as f:
    while True:
        bytes = f.read(4*buffer_size)
        if len(bytes) == 0:
            break
        buffer_values = unpack('i'*(len(bytes)/4), bytes)
        for idx, value in enumerate(buffer_values):
            if value <= 0:
                last_node = -value
                cnt += 1
                if cnt%100000==0:
                    print last_node, cnt
            else:
                memcount[value] += 1
        del(bytes)

113737 100000
228876 200000
344079 300000
463516 400000
583321 500000
698462 600000
815079 700000
931946 800000
1048433 900000
1165598 1000000
1282171 1100000
1413511 1200000
1529700 1300000
1645369 1400000
1763036 1500000
1880112 1600000
1996892 1700000
2113407 1800000
2230177 1900000
2345467 2000000
2461750 2100000
2585634 2200000
2740121 2300000
2989358 2400000
3211816 2500000
3381999 2600000
3518242 2700000
3639270 2800000
3762591 2900000
3880464 3000000
3998185 3100000
4115595 3200000
4233245 3300000
4351303 3400000
4469163 3500000
4587888 3600000
4706218 3700000
4824933 3800000
4943667 3900000
5060849 4000000
5177508 4100000
5293963 4200000
5410966 4300000
5528091 4400000
5645439 4500000
5763242 4600000
5881488 4700000
5999614 4800000
6117517 4900000
6235833 5000000
6354061 5100000
6471337 5200000
6588210 5300000
6706344 5400000
6823931 5500000
6941568 5600000
7057911 5700000
7177599 5800000
7297283 5900000
7418104 6000000
7550722 6100000
7669031 6200000
7794919 6300000
7913560 6

In [ ]:
np.save('D:\\vk_1015\\memcount',memcount)

In [ ]:
fillval = pack('i', 0)
with open(invfname, 'wb') as f:
    for uid, cnt in enumerate(memcount):
        if cnt > 0:
            f.write(pack('i', -uid))
            for i in range(int(cnt)):
                f.write(fillval)

In [ ]:
findex = np.zeros(n_nodes, dtype=np.int)
pos = 1
for uid, cnt in enumerate(memcount):
    findex[uid] = pos
    if cnt > 0:
        pos += cnt + 1

In [ ]:
print findex[:10]

In [ ]:
buffer_size = 1024
last_node = 1
cnt = 0
with open(invfname, 'r+') as outf:
    with open(fname, 'rb') as f:
        while True:
            bytes = f.read(4*buffer_size)
            if len(bytes) == 0:
                break
            buffer_values = unpack('i'*(len(bytes)/4), bytes)
            for idx, value in enumerate(buffer_values):
                if value <= 0:
                    last_node = -value
                    cnt += 1
                    if cnt%100000==0:
                        print last_node, cnt
                else:
                    if value == 0:
                        print 'writing', last_node, 'to', findex[value] * 4, 'value', value
                    if findex[value] * 4 == 0:
                        print 'writing', last_node, 'to', findex[value] * 4, 'value', value
                    outf.seek(findex[value] * 4, 0)
                    findex[value] += 1
                    outf.write(pack('i', last_node))
            del(bytes)

In [ ]:
buffer_size = 1024
last_node = 1
cnt = 0
with open(invfname, 'rb') as f:
    while True:
        bytes = f.read(4*buffer_size)
        if len(bytes) == 0:
            break
        buffer_values = unpack('i'*(len(bytes)/4), bytes)
        print buffer_values
        break